# Using Schemas with LLaMa
## Goal get a python object back from LLaMa
1. Define a pydantic schema
2. Define a langchain series of prompts
3. Get a python object in return

In [ ]:
!pip install langchain-groq langchain

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [ ]:
from google.colab import userdata
model = ChatGroq(temperature=0.1, groq_api_key=userdata.get('GROQ_API_KEY'), model_name="llama2-70b-4096")

In [ ]:
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator

In [ ]:
# Here's another example, but with a compound typed field.
class Animal(BaseModel):
    name: str = Field(description="type of animal")
    likes_to_eat: List[str] = Field(description="list of foods it likes")


animal_query = "Generate what foods a random animal likes to eat. Make it a bird."

parser = PydanticOutputParser(pydantic_object=Animal)

prompt = PromptTemplate(
    template="Generate only the JSON file.  \n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


In [ ]:
chain = prompt | model | parser

animal = chain.invoke({"query": animal_query})

In [ ]:
animal.name

'Blue Jay'

In [ ]:
animal.likes_to_eat

['seeds', 'nuts', 'fruits', 'insects']